In [22]:
from keras_contrib.layers.normalization import InstanceNormalization, BatchRenormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, merge, Deconv2D, Add
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import ReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.engine.topology import Layer
from keras.engine import InputSpec
import tensorflow as tf

In [23]:
class ReflectionPadding2D(Layer):
    """
    from https://stackoverflow.com/questions/50677544/reflection-padding-conv2d
    """
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def get_output_shape_for(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

In [24]:
width          = 1024
height         = 1024
input_channel  = 2
output_channel = 1
image_shape    = (width, height, input_channel)

In [29]:
        def ck(layer_input, filters, f_size=7, i_norm=True) :
            d = ReflectionPadding2D(padding=(3, 3))(layer_input)
            d = Conv2D(filters, kernel_size=f_size, strides=1, padding="same")(d)
            if i_norm == True :
                d = InstanceNormalization()(d)
            d = ReLU()(d)
            return d
        def dk(layer_input, filters, f_size=3, i_norm=True) :
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding="same")(layer_input)
            if i_norm == True :
                d = InstanceNormalization()(d)
            d = ReLU()(d)
            return d
        def Rk(layer_input, filters, f_size=3, i_norm = True) :
            d = ReflectionPadding2D(padding=(1, 1))(layer_input)
            d = Conv2D(filters, kernel_size=f_size, padding="valid")(d)

            if i_norm == True :
                d = InstanceNormalization()(d)
            d = ReLU()(d)
            d = ReflectionPadding2D(padding=(1, 1))(d)
            d = Conv2D(filters, kernel_size=f_size, padding="valid")(d)
            if i_norm == True :
                d = InstanceNormalization()(d)
            d = layer_input + d
            d = ReLU()(d)
            return d
        def uk(layer_input, filters, f_size=3, i_norm=True) :
            d = Deconv2D(filters, kernel_size=f_size, padding="valid")(layer_input)
            if i_norm == True:
                d = InstanceNormalization()(d)
            d = ReLU()(d)
            return d

In [32]:
gn_shape = (image_shape[0] / 2, image_shape[1] / 2, image_shape[2])

g0 = Input(shape=gn_shape)
g1 = ck(g0, 64)
g2 = dk(g1, 128)
g3 = dk(g2, 256)
g4 = dk(g3, 512)
g5 = dk(g4, 1024)
g6 = Rk(g5, 1024)
g7 = Rk(g6, 1024)
g8 = Rk(g7, 1024)
g9 = Rk(g8, 1024)
g10 = Rk(g9, 1024)
g11 = Rk(g10, 1024)
g12 = Rk(g11, 1024)
g13 = Rk(g12, 1024)
print(g13)
g14 = Rk(g13, 1024)
print(g14)
g15 = uk(g14, 512)
print(g15)
g16 = uk(g15, 256)
print(g16)
g17 = uk(g16, 128)
print(g17)
g18 = uk(g17, 64)
print(g18)

Tensor("re_lu_125/Relu:0", shape=(?, 33, 33, 1024), dtype=float32)
Tensor("re_lu_127/Relu:0", shape=(?, 33, 33, 1024), dtype=float32)
Tensor("re_lu_128/Relu:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("re_lu_129/Relu:0", shape=(?, ?, ?, 256), dtype=float32)
Tensor("re_lu_130/Relu:0", shape=(?, ?, ?, 128), dtype=float32)
Tensor("re_lu_131/Relu:0", shape=(?, ?, ?, 64), dtype=float32)


In [33]:
gn_output = ck(g18, 1)

In [34]:
print(gn_output)

Tensor("re_lu_132/Relu:0", shape=(?, ?, ?, 1), dtype=float32)


In [35]:
Model(g0, gn_output)

AttributeError: 'NoneType' object has no attribute '_inbound_nodes'

In [45]:
def Ck(input_layer, filters, f_size=4, i_norm=True) :
    d = Conv2D(filters, kernel_size=f_size, strides=2, padding="same")(input_layer)
    if i_norm == True:
        d = InstanceNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    return d
d0 = Input((1024, 1024, 2))
d1 = Ck(d0, 64, i_norm=False)
d2 = Ck(d1, 128)
d3 = Ck(d2, 256)
d4 = Ck(d3, 512)
output_prediction = Conv2D(1, kernel_size=7, strides=9 , padding="same")(d4)
model = Model(d0, output_prediction)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 1024, 1024, 2)     0         
_________________________________________________________________
conv2d_164 (Conv2D)          (None, 512, 512, 64)      2112      
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 512, 512, 64)      0         
_________________________________________________________________
conv2d_165 (Conv2D)          (None, 256, 256, 128)     131200    
_________________________________________________________________
instance_normalization_161 ( (None, 256, 256, 128)     2         
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 256, 256, 128)     0         
_________________________________________________________________
conv2d_166 (Conv2D)          (None, 128, 128, 256)     524544    
__________